# Prerequisites

In [ ]:
import pandas as pd
import numpy as np
import os
import openpyxl

In [ ]:
data = pd.read_csv('today.csv')

C:\Users\harsh\AppData\Local\Temp\ipykernel_5512\1851084234.py:1: DtypeWarning: Columns (51,59,61,62,63,64,71,72,73,75,76,77,83,85,90,95,96,99,104,109,112) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('today.csv')


,Product,Vessel__Strategy,Unit,OPS Unit,ORSUnit,Date,Trade/OPS Ref,Party,Party Ref,Grade,...,Broker,Broker Reference,Party Code,Fixing Date,Grade Optionality,Design,Provisional Price,BL Date,BL Quantity,Outport
0,ANIPRO,000 - UnLinked,CFM,CFM,CFM,2025-04-30 00:00:00,5141000011693,NaN,NaN,NaN,...,NaN,NaN,NaN,2025-05-30 00:00:00,NaN,0,NaN,NaN,NaN,NaN
1,ANIPRO,000 - UnLinked,CFM,CFM,CFM,2025-04-30 00:00:00,5141000011833,NaN,NaN,NaN,...,NaN,NaN,NaN,2025-05-30 00:00:00,NaN,0,NaN,NaN,NaN,NaN
2,ANIPRO,000 - UnLinked,CFM,CFM,CFM,2025-04-30 00:00:00,5141000011936,NaN,NaN,NaN,...,NaN,NaN,NaN,2025-05-30 00:00:00,NaN,0,NaN,NaN,NaN,NaN
3,ANIPRO,000 - UnLinked,CFM,CFM,CFM,2025-04-30 00:00:00,5141000011963,NaN,NaN,NaN,...,NaN,NaN,NaN,2025-05-30 00:00:00,NaN,0,NaN,NaN,NaN,NaN
4,ANIPRO,000 - UnLinked,CFM,CFM,CFM,2025-04-30 00:00:00,5141000012128,NaN,NaN,NaN,...,NaN,NaN,NaN,2025-05-30 00:00:00,NaN,0,NaN,NaN,NaN,NaN


## Cleaning the File

### Date Formatting

In [ ]:
cols = ['Date', 'Shipment Month', 'Terminal Month', 'M2M Terminal Month']
for i in cols:
    formatted_dates = pd.to_datetime(data[i], format='mixed', errors='coerce')
    data[i] = formatted_dates

,Product,Vessel__Strategy,Unit,OPS Unit,ORSUnit,Date,Trade/OPS Ref,Party,Party Ref,Grade,...,Broker,Broker Reference,Party Code,Fixing Date,Grade Optionality,Design,Provisional Price,BL Date,BL Quantity,Outport
0,ANIPRO,000 - UnLinked,CFM,CFM,CFM,2025-04-30,5141000011693,NaN,NaN,NaN,...,NaN,NaN,NaN,2025-05-30 00:00:00,NaN,0,NaN,NaN,NaN,NaN
1,ANIPRO,000 - UnLinked,CFM,CFM,CFM,2025-04-30,5141000011833,NaN,NaN,NaN,...,NaN,NaN,NaN,2025-05-30 00:00:00,NaN,0,NaN,NaN,NaN,NaN
2,ANIPRO,000 - UnLinked,CFM,CFM,CFM,2025-04-30,5141000011936,NaN,NaN,NaN,...,NaN,NaN,NaN,2025-05-30 00:00:00,NaN,0,NaN,NaN,NaN,NaN
3,ANIPRO,000 - UnLinked,CFM,CFM,CFM,2025-04-30,5141000011963,NaN,NaN,NaN,...,NaN,NaN,NaN,2025-05-30 00:00:00,NaN,0,NaN,NaN,NaN,NaN
4,ANIPRO,000 - UnLinked,CFM,CFM,CFM,2025-04-30,5141000012128,NaN,NaN,NaN,...,NaN,NaN,NaN,2025-05-30 00:00:00,NaN,0,NaN,NaN,NaN,NaN


# Making Pivot Tables

## Pivots for Groups Of Units

In [5]:
group = input("Enter the units you want to group (comma separated): ")
group_list = group.split(',')
group = [i.strip() for i in group_list]
print(f"Grouping the following units: {group}")
group_data = data[data['Unit'].isin(group_list)]
name = input("Enter a name for the group: ")
os.makedirs(f"Group Files/{name}", exist_ok=True)
group_data.to_csv(f"Group Files/{name}/unit_data_today.csv", index=False)

Grouping the following units: ['GCB', 'GR-BSC', 'GR-SPAIN']


### Outright for Group

In [ ]:
unit_data = pd.read_csv(f"Group Files/{name}/unit_data_today.csv")
filter_data = unit_data[~unit_data['Trade Type'].isin(['FX','FX_BK'])]
pivot_table = filter_data.pivot_table(
    columns='M2M Terminal Month',
    index='Trade Type',
    values='Qty Exposure',
    aggfunc='sum',
    margins=True,
    margins_name='Grand Total',
)
outright_pivot_table = pivot_table.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else "0")

C:\Users\harsh\AppData\Local\Temp\ipykernel_5512\11891516.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  outright_pivot_table = pivot_table.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else 0)


### Basis Position for Each Group

In [ ]:
unit_data = pd.read_csv(f"Group Files/{name}/unit_data_today.csv")
filter_data = unit_data[unit_data['Transaction Type'].isin(['PHY'])]
pivot_table = filter_data.pivot_table(
    columns='Shipment Month',
    index='Trade Type',
    values='Delta Quantity',
    aggfunc='sum',
    margins=True,
    margins_name='Grand Total',
)
basis_pivot_table = pivot_table.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else "0")

C:\Users\harsh\AppData\Local\Temp\ipykernel_5512\2766865501.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basis_pivot_table = pivot_table.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else 0)


### M2M Currency Risk for Each Group

In [ ]:
unit_data = pd.read_csv(f"Group Files/{name}/unit_data_today.csv")
filter_data = unit_data[unit_data['Transaction Type'].isin(['PHY'])]
pivot_table = filter_data.pivot_table(
    columns='Shipment Month',
    index='Trade Type',
    values='M2M Currency Risk',
    aggfunc='sum',
    margins=True,
    margins_name='Grand Total',
)
m2mcurr_pivot_table = pivot_table.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else "0")

C:\Users\harsh\AppData\Local\Temp\ipykernel_5512\1434519205.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  m2mcurr_pivot_table = pivot_table.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else 0)


### Euro Exposure for Each Group

In [9]:
unit_data = pd.read_csv(f"Group Files/{name}/unit_data_today.csv")

# Filter data for Euro Exposure
filter_data = unit_data[(unit_data['Contract Currency'] == 'EUR') & (unit_data['Transaction Type'] == 'PHY')]
filter_data2 = unit_data[(unit_data['Exch'] == 'EOP') & (unit_data['Trade Type'].isin(['FUT', 'CALL', 'PUT']))]
filter_data3 = unit_data[(unit_data['Contract Currency'] == 'EUR') & (unit_data['Trade Type']== 'FX')]
combined = pd.concat([filter_data, filter_data2, filter_data3], ignore_index=True)
combined_unique = combined.drop_duplicates()
combined_unique["Exposure"] = filter_data['Delta Quantity'] * filter_data['Price'] * -1


pivot_table = combined_unique.pivot_table(
    columns='Shipment Month',
    index='Trade Type',
    values='Exposure',
    aggfunc='sum',
    margins=True,
    margins_name='Grand Total',
)
eurexp_pivot_table = pivot_table.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else "0")

C:\Users\harsh\AppData\Local\Temp\ipykernel_5512\1837864726.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  eurexp_pivot_table = pivot_table.applymap(lambda x: f"{x:,.0f}" if pd.notnull(x) else "0")


In [10]:
wb = openpyxl.Workbook()
excel_path = f"Group Files/{name}/Position_Pivots.xlsx"

with pd.ExcelWriter(excel_path,engine='openpyxl') as writer:

    outright_pivot_table.to_excel(writer,sheet_name='Outright Position')
    basis_pivot_table.to_excel(writer,sheet_name='Basis Position')
    m2mcurr_pivot_table.to_excel(writer,sheet_name='M2M Currency Risk Position')
    eurexp_pivot_table.to_excel(writer,sheet_name='Euro Exposure Position')